# Basic Example of a SLP and MLP for Classification and Regression

This is a basic example of a Single and Multilayer Perceptron for Classification and Regression based on keras.
It also uses sklearn for auxiliar functions.

The general scheme is as follows:
- Set the values of the global variables
- Load data
- Preprocess data
- Split data into training and test
- Create the architecture
- Select the elements previous to the training phase: loss function, training algorithm, ...
- Train the system with the training data
- Test the model

Keras documentation: https://keras.io/api

Sklearn documentation: https://scikit-learn.org/stable/user_guide.html

**This script has been tested with the following package versions:**
- pandas 1.3.3
- sklearn 0.24.0
- keras 2.2.4 + tensorflow 1.14.0 / keras 2.6.0 + tensorflow 2.6.0

## General imports

In [ ]:
import DataFunctions                ### Functions for data management
import numpy                        ### Library for numerical computations
import keras, tensorflow, sklearn   ### Libraries for constructing and training the models
import matplotlib.pyplot as plt     ### Library for plotting

In [ ]:
print(keras.__version__)
print(tensorflow.__version__)

## Global variables for the script

In [ ]:
### First we set the values of several global variables
MULTILAYER_PERCEPTRON = False  ### If False, it is a Single-layer Perceptron
CLASSIFICATION        = True   ### If False, it is a REGRESSION problem

## Load inputs and labels

In [ ]:
### Now we read inputs and labels
inputsFileName   = 'Data/ionosphere.inputs'
labelsFileName = 'Data/ionosphere.labels'
#inputsFileName   = 'Data/hepatitis.inputs'
#labelsFileName = 'Data/hepatitis.labels'
#inputsFileName   = 'Data/sonar.inputs'
#labelsFileName = 'Data/sonar.labels'
x, y = DataFunctions.loadDatasetsFromFiles (inputsFileName, labelsFileName)
y    = y.ravel()   ### sklearn prefers shapes (N,) than (N,1)

nExamples = x.shape[0]
nFeatures = x.shape[1]
if CLASSIFICATION:
    nClasses  = len(numpy.unique(y))

## Convert labels to a 1-of-C (one-hot) scheme

In [ ]:
### For neural networks, it is easier to output yes/no than (for example) an integer with the predicted class
if CLASSIFICATION:
    y1C  = DataFunctions.convertLabels_1ofC_Scheme (y)

#### Print Some Information about inputs and labels

In [ ]:
### Type, dimensions, number of examples in every class, first rows, ...
print("Type of variable x: %s" % type(x))
print("Type of variable y: %s" % type(y))
print("Dimensions of variable   x: %3d %3d  " % x.shape,   end=""); print("  shape:",x.shape)
print("Dimensions of variable   y: %3d      " % y.shape,   end=""); print("  shape:",y.shape)
if CLASSIFICATION:
    print("Dimensions of variable y1C: %3d %3d  " % y1C.shape, end=""); print("  shape:",y1C.shape)
    print("Number of examples in every class:")
    for i in numpy.unique(y):
        print ("  class %d: %d" % (i,sum(y==i)))
print("First 3 rows of x:"); print(x[0:3,])
print("First 3 rows of y:"); print(y[0:3])
if CLASSIFICATION:
    print("First 3 rows of y1C:"); print(y1C[0:3,])

## Scale inputs

In [ ]:
### Now we scale the inputs
x, Scaler = DataFunctions.scaleDataMean0Dev1Scaler (x)
#x, Scaler = DataFunctions.scaleDataMinMaxScaler (x, FeatureRange=(-1,+1))
print("First 3 rows of x:"); print(x[0:3,])

## Split data and labels into training and test data

In [ ]:
### Split data into training (to construct the model) and test (to estimate the generalization)
from sklearn import model_selection
x_train, x_test, y_train, y_test = \
  model_selection.train_test_split (x, y, train_size=0.70, shuffle=True, stratify=y)
if CLASSIFICATION:
    y1C_train = DataFunctions.convertLabels_1ofC_Scheme (y_train)
    y1C_test  = DataFunctions.convertLabels_1ofC_Scheme (y_test)

#### Print some information about training and test data

In [ ]:
### Same as above
print("Dimensions of variable   x_train: %3d %3d " % x_train.shape, end="");   print("  shape:",x_train.shape)
print("Dimensions of variable   y_train: %3d     " % y_train.shape, end="");   print("  shape:",y_train.shape)
if CLASSIFICATION:
    print("Dimensions of variable y1C_train: %3d %3d " % y1C_train.shape, end=""); print("  shape:",y1C_train.shape)
    print("Number of examples in every class in x_train:")
    for i in numpy.unique(y_train):
        print ("  class %d: %3d" % (i,sum(y_train==i)))
print("Dimensions of variable   x_test: %3d %3d " % x_test.shape, end="");   print("  shape:",x_test.shape)
print("Dimensions of variable   y_test: %3d     " % y_test.shape, end="");   print("  shape:",y_test.shape)
if CLASSIFICATION:
    print("Dimensions of variable y1C_test: %3d %3d " % y1C_test.shape, end=""); print("  shape:",y1C_test.shape)
    print("Number of examples in every class in x_test:")
    for i in numpy.unique(y_test):
        print ("  class %d: %3d" % (i,sum(y_test==i)))

## Create the architecture (Type of Problem + Model Representation)

In [ ]:
###
### https://keras.io/api/models/sequential/#sequential-class
###

### First we indicate that it is a sequential model
myNetwork = keras.Sequential()

if CLASSIFICATION:
    nOutput           = nClasses
    fActivationOutput = 'softmax'
else:
    nOutput           = 1
    fActivationOutput = 'linear'

### We need to indicate the input dimension in the first layer
inputDimension = nFeatures

if MULTILAYER_PERCEPTRON:

    ### Now we add the hidden layers
    nHidden1     = 100
    fActivation1 = 'tanh'
    myNetwork.add ( keras.layers.Dense (nHidden1, activation=fActivation1, input_dim=inputDimension) )
    #myNetwork.add ( keras.layers.Dense (nHidden1, activation=fActivation1, kernel_regularizer=keras.regularizers.l2(0.01), bias_regularizer=keras.regularizers.l2(0.01), input_dim=nFeatures) )
    nHidden2     = 50
    fActivation2 = 'tanh'
    myNetwork.add ( keras.layers.Dense (nHidden2, activation=fActivation2) )
    #myNetwork.add ( keras.layers.Dense (nHidden2, activation=fActivation2, kernel_regularizer=keras.regularizers.l2(0.01), bias_regularizer=keras.regularizers.l2(0.01)) )

    ### And finally we add the output layer
    myNetwork.add ( keras.layers.Dense (nOutput, activation=fActivationOutput) )

else:

    ### We only have an output layer
    myNetwork.add ( keras.layers.Dense (nOutput, activation=fActivationOutput, input_dim=inputDimension) )

### Print statistics
print(myNetwork.summary())

### Now we create a keras model
myInput = keras.layers.Input (shape=(nFeatures,))
myModel = keras.models.Model (inputs=myInput, outputs=myNetwork(myInput))

## Select a loss function (Type of Problem + Cost Function)

In [ ]:
### Usual loss functions: 'categorical_crossentropy' 'binary_crossentropy' 'mean_squared_error', etc
if CLASSIFICATION:
    lossFunction = ['categorical_crossentropy']  # For one-hot labels, use categorical_crossentropy
else:
    lossFunction = ['mean_squared_error']

## Select a training algorithm and its parameters (Optimization Technique)

In [ ]:
### Every training algorithm will have its own parameters
LearningRate = 0.01
Momentum     = 0.8
#print(keras.__version__)
if keras.__version__ == "2.2.4":
    optimizers = keras.optimizers
else:
    optimizers = tensorflow.keras.optimizers
trainAlgorithm = optimizers.SGD (lr=LearningRate, momentum=Momentum)  # There are more parameters
#
#LearningRate = 0.01
#trainAlgorithm = optimizers.RMSprop (lr=LearningRate)                 # There are more parameters
#
#LearningRate = 0.01
#trainAlgorithm = optimizers.Adam (lr=LearningRate)                    # There are more parameters

## Select the metrics we want to monitorize

In [ ]:
### Keras allows to monitorize several metrics along training
if CLASSIFICATION:
    showMetrics = ['categorical_accuracy', 'mean_squared_error']  # For one-hot labels, use categorical_accuracy
else:
    showMetrics = ['mean_squared_error']

## Compile the model with all the elements

In [ ]:
### This is the standard way to work in keras
myModel.compile (loss=lossFunction, optimizer=trainAlgorithm, metrics=showMetrics)

## Train the model with the training data

In [ ]:
###
### This method has many parameters:
###   https://keras.io/api/models/model_training_apis/#fit-method
###

batchSize = 20
nEpochs   = 200
if CLASSIFICATION:
    validationData = (x_test,y1C_test)  ### We could also use the validation_split parameter
    fitData = myModel.fit \
      (x_train, y1C_train, validation_data=validationData, batch_size=batchSize, epochs=nEpochs)
else:
    validationData = (x_test,y_test)    ### We could also use the validation_split parameter
    fitData = myModel.fit \
      (x_train, y_train,   validation_data=validationData, batch_size=batchSize, epochs=nEpochs)

## Test the model in the training and test data at the end of the training phase

In [ ]:
if CLASSIFICATION:
    scoresTrain = myModel.evaluate (x_train,y1C_train)
    scoresTest  = myModel.evaluate (x_test,y1C_test)
    print("Loss function and Accuracy in the training set: %.8f  %7.3f%%" % (scoresTrain[0], 100*scoresTrain[1])) 
    print("Loss function and Accuracy in the test set:     %.8f  %7.3f%%" % (scoresTest[0],  100*scoresTest[1]))
else:
    scoresTrain = myModel.evaluate (x_train,y_train)
    scoresTest  = myModel.evaluate (x_test,y_test)
    print("Loss function and Squared Error in the training set: %.8f  %.8f" % (scoresTrain[0], scoresTrain[1])) 
    print("Loss function and Squared Error in the test set:     %.8f  %.8f" % (scoresTest[0],  scoresTest[1]))

In [ ]:
print(type(fitData)); print(fitData); print("---")
print(dir(fitData)); print("---")
print(type(fitData.history)); print(fitData.history.keys());

## Now we can plot the training history

In [ ]:
lossTrain = fitData.history["loss"]
lossValid = fitData.history["val_loss"]

epochsPlot = range(1,len(lossTrain)+1)

plt.plot(epochsPlot,lossTrain,label='Training Loss')
plt.plot(epochsPlot,lossValid,label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
if CLASSIFICATION:
    accuracyTrain = fitData.history["categorical_accuracy"]
    accuracyValid = fitData.history["val_categorical_accuracy"]

    epochsPlot = range(1,len(accuracyTrain)+1)

    plt.plot(epochsPlot,accuracyTrain,label='Training Accuracy')
    plt.plot(epochsPlot,accuracyValid,label='Validation Accuracy')

    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()